In [3]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np

In [54]:
nb_nodes = 5
nb_edges = 3

In [65]:
def gt_fn(x, nb_nodes):
    x = x.reshape(-1,nb_nodes)
    return (np.max(x[:,[0,1]], axis=1) > ((x[:,2]) + x[:,3]/2.0))*1.0
gt_fn(np.random.rand(2,1,nb_nodes), nb_nodes)

array([ 0.,  1.])

In [66]:
X_train = np.random.rand(1000,nb_nodes)*10
y_train = gt_fn(X_train, nb_nodes)

X_test = np.random.rand(1000,nb_nodes)*10
y_test = gt_fn(X_test, nb_nodes)

In [69]:
import sklearn
def sample(X_batch, y_batch, size, num_classes):
    
    X_batch = np.asarray(X_batch)
    y_batch = np.asarray(y_batch)
    
    x_data = []
    y_data = []
    
    for i in range(num_classes):
        x_data.append(sklearn.utils.resample(X_batch[np.where(y_batch == i)[0]], n_samples=size))
        y_data.append(np.zeros((size, 1))+i)


    X_batch = np.concatenate(x_data)
    y_batch = np.concatenate(y_data)

    todo = range(len(y_batch))
    np.random.shuffle(todo)

    X_batch = X_batch[todo]
    y_batch = y_batch[todo]
    
    return X_batch, y_batch

sample(X_train, y_train, 2,2)

(array([[ 7.72451533,  7.91903082,  9.22116484,  0.15166088,  1.94498966],
        [ 8.7556352 ,  5.01779175,  9.57385972,  1.09987627,  1.40066659],
        [ 8.98780765,  8.75884816,  2.74960823,  7.68339889,  0.88637747],
        [ 1.86853918,  7.14388589,  0.14656092,  9.97961843,  2.96131355]]),
 array([[ 0.],
        [ 0.],
        [ 1.],
        [ 1.]]))

In [70]:
# Create a lambda graph with adjancy matrix and stuff.
def random_adj(nb_nodes, nb_edges):
    # nodes
    nodes = np.arange(nb_nodes)

    # roughly nb_edges edges
    edges = np.array([(i, ((((i + np.random.randint(nb_nodes - 1))  % nb_nodes) + 1 ) % nb_nodes ))
                      for i in [np.random.randint(nb_nodes) for i in range(nb_edges)]])

    # Adding self loop.
    edges = np.concatenate((edges, np.array([(i, i) for i in nodes])))


    # adjacent matrix
    A = np.zeros((nb_nodes, nb_nodes))
    A[edges[:, 0], edges[:, 1]] = 1.
    A[edges[:, 1], edges[:, 0]] = 1.

    # Degree matrix
    D = A.sum(axis=1)
    
    return A, D

A, D = random_adj(nb_nodes, nb_edges)

In [71]:
A

array([[ 1.,  0.,  0.,  1.,  0.],
       [ 0.,  1.,  0.,  0.,  1.],
       [ 0.,  0.,  1.,  1.,  0.],
       [ 1.,  0.,  1.,  1.,  0.],
       [ 0.,  1.,  0.,  0.,  1.]])

In [72]:
edges_np = np.asarray(np.where(A == 1)).T
#edges_np = np.array([(0,3),(3,1),(1,2),(2,3)])
#edges_np = np.concatenate([edges_np,edges_np[:,[1,0]]])

In [73]:
edges_np

array([[0, 0],
       [0, 3],
       [1, 1],
       [1, 4],
       [2, 2],
       [2, 3],
       [3, 0],
       [3, 2],
       [3, 3],
       [4, 1],
       [4, 4]])

In [74]:
edge_selector = np.array([np.where(edges_np[:,0] == i)[0] for i in range(4)])

In [75]:
edge_selector

array([array([0, 1]), array([2, 3]), array([4, 5]), array([6, 7, 8])], dtype=object)

In [76]:
edges = torch.LongTensor(edges_np)

In [77]:
edges.contiguous()


    0     0
    0     3
    1     1
    1     4
    2     2
    2     3
    3     0
    3     2
    3     3
    4     1
    4     4
[torch.LongTensor of size 11x2]

In [78]:
weights = Variable(torch.rand(edges.shape), requires_grad=True)

In [79]:
weights

Variable containing:
 0.3072  0.3498
 0.7072  0.4577
 0.3898  0.2200
 0.4454  0.9831
 0.7896  0.1396
 0.7992  0.6529
 0.8976  0.8330
 0.4447  0.5398
 0.8860  0.0977
 0.9678  0.5982
 0.8675  0.2229
[torch.FloatTensor of size 11x2]

In [80]:
edges.contiguous().view(-1)


 0
 0
 0
 3
 1
 1
 1
 4
 2
 2
 2
 3
 3
 0
 3
 2
 3
 3
 4
 1
 4
 4
[torch.LongTensor of size 22]

In [239]:
Variable(torch.FloatTensor(X_train[:2]))

Variable containing:
 1.6451  0.8749  4.0963  2.2694  8.5953
 7.2118  1.5155  5.3553  9.9597  6.6334
[torch.FloatTensor of size 2x5]

In [243]:
Variable(torch.FloatTensor(X_train[:2])).view(2,-1,nb_nodes)

Variable containing:
(0 ,.,.) = 
  1.6451  0.8749  4.0963  2.2694  8.5953

(1 ,.,.) = 
  7.2118  1.5155  5.3553  9.9597  6.6334
[torch.FloatTensor of size 2x1x5]

In [244]:
x = Variable(torch.FloatTensor(X_train[:2])).view(2,-1,nb_nodes)
x = x[:,0,:]
tocompute = torch.index_select(x, 1, Variable(edges.contiguous().view(-1))).view(2,-1,2)
tocompute

Variable containing:
(0 ,.,.) = 
  1.6451  1.6451
  1.6451  2.2694
  0.8749  0.8749
  0.8749  8.5953
  4.0963  4.0963
  4.0963  2.2694
  2.2694  1.6451
  2.2694  4.0963
  2.2694  2.2694
  8.5953  0.8749
  8.5953  8.5953

(1 ,.,.) = 
  7.2118  7.2118
  7.2118  9.9597
  1.5155  1.5155
  1.5155  6.6334
  5.3553  5.3553
  5.3553  9.9597
  9.9597  7.2118
  9.9597  5.3553
  9.9597  9.9597
  6.6334  1.5155
  6.6334  6.6334
[torch.FloatTensor of size 2x11x2]

In [138]:
tocompute*weights

Variable containing:
 0.5054  0.5754
 1.1634  1.0387
 0.3411  0.1924
 0.3897  8.4505
 3.2343  0.5717
 3.2739  1.4816
 2.0369  1.3704
 1.0091  2.2111
 2.0106  0.2217
 8.3189  0.5234
 7.4566  1.9156
[torch.FloatTensor of size 11x2]

In [247]:
class CGN2(nn.Module):
    def __init__(self,input_dim, A, channels=1, out_dim=2, on_cuda=False):
        super(CGN2, self).__init__()

        self.my_layers = []
        self.out_dim = out_dim
        self.on_cuda = on_cuda
        
        self.channels = channels
        #dims = [input_dim] + channels
        
        print "Constructing the network..."   
        
        
        edges_np = np.asarray(np.where(A == 1)).T
        self.edges = torch.LongTensor(edges_np)
        
        self.weights1 = nn.Parameter(torch.rand(self.edges.shape), requires_grad=True)
        print self.weights1.size()

        self.last_layer = nn.Linear(input_dim, out_dim)
        self.my_layers = nn.ModuleList([self.last_layer])

        
        
        
        self.edge_selector = np.array([np.where(edges_np[:,0] == i)[0] for i in range(input_dim)])
        print "edge_selector", self.edge_selector
        
        
        
        
        
        print "Done!"


    #print x
    def GraphConv(self, x, edges, channel, batch_size, weights):
        
        x = x.clone()
        #x = x.view(batch_size, -1)
        x = x[:,channel,:]
        tocompute = torch.index_select(x, 1, Variable(edges.contiguous().view(-1))).view(batch_size, -1, 2)
        #print tocompute
        conv = tocompute*weights
        #print conv
        for i, edges_to_select in enumerate(self.edge_selector):
            #print "x", conv
            #print "e", edges_to_select
            selected_edges = torch.index_select(conv, 1, Variable(torch.LongTensor(edges_to_select)))
            #print "m", selected_edges
            selected_edges = selected_edges.view(-1,edges_to_select.shape[0]*2)
            #print "m", selected_edges
            pooled_edges = torch.max(selected_edges,1)[0]
            #print "mmo",pooled_edges
            x[:,i] = pooled_edges
            #print "xx",x[:,i]
        return x

        
        
    def forward(self, x):
        nb_examples, nb_nodes, nb_channels = x.size()
        
        #print x
        self.GraphConv(x, self.edges, 0, nb_examples, self.weights1)

        x = self.last_layer(x.view(nb_examples, -1))
        x = F.softmax(x)

        return x

In [248]:
import models
reload(models);

In [249]:
gc = CGN2(nb_nodes,A)
#gc = models.CGN2(4,edges)
#gc = models.MLP(4,[10],2)
gc

Constructing the network...
torch.Size([11, 2])
edge_selector [array([0, 1]) array([2, 3]) array([4, 5]) array([6, 7, 8]) array([ 9, 10])]
Done!


CGN2 (
  (last_layer): Linear (5 -> 2)
  (my_layers): ModuleList (
    (0): Linear (5 -> 2)
  )
)

In [250]:
list(gc.parameters())

[Parameter containing:
  0.4419  0.0523
  0.2943  0.2016
  0.1141  0.0607
  0.7146  0.6348
  0.2847  0.9168
  0.0423  0.1458
  0.7514  0.1360
  0.0250  0.4984
  0.9948  0.4843
  0.2092  0.7809
  0.7862  0.6508
 [torch.FloatTensor of size 11x2], Parameter containing:
  0.1273  0.3478  0.3440  0.2377 -0.3365
  0.0495  0.3343  0.4319 -0.4356  0.3580
 [torch.FloatTensor of size 2x5], Parameter containing:
 -0.0285
 -0.1435
 [torch.FloatTensor of size 2]]

In [251]:
optimizer = optim.SGD(gc.parameters(), lr = 0.001, momentum=0.9)

In [252]:
X_batch, y_batch = sample(X_train, y_train, 3,2)

In [277]:
x = Variable(torch.FloatTensor(X_batch)).view(-1,1,nb_nodes)
x

Variable containing:
(0 ,.,.) = 
  4.6067  8.7310  4.3838  5.2321  3.2312

(1 ,.,.) = 
  0.7580  6.3974  0.9436  6.6440  3.5297

(2 ,.,.) = 
  0.7398  7.2101  2.6227  0.4762  3.1642

(3 ,.,.) = 
  0.5058  3.9238  7.0303  6.1787  5.9759

(4 ,.,.) = 
  3.6904  4.6796  2.6793  9.3792  8.1080

(5 ,.,.) = 
  3.6904  4.6796  2.6793  9.3792  8.1080
[torch.FloatTensor of size 6x1x5]

In [278]:
output = gc(x)[:,1]

In [279]:
output

Variable containing:
 0.1472
 0.0791
 0.8479
 0.5101
 0.2033
 0.2033
[torch.FloatTensor of size 6]

In [280]:
gt = Variable(torch.FloatTensor(y_batch), requires_grad=False).view(-1)
loss = torch.abs(gt-output)
print loss
print loss.mean()

Variable containing:
 0.8528
 0.9209
 0.1521
 0.5101
 0.2033
 0.2033
[torch.FloatTensor of size 6]

Variable containing:
 0.4737
[torch.FloatTensor of size 1]



In [281]:
loss.mean().backward()
optimizer.step()

In [705]:
X_batch.shape

(2, 4)

In [706]:
x = Variable(torch.FloatTensor(X_batch))
output = gc(x)
#gt = Variable(torch.FloatTensor(gt_fn(x.data.numpy()))).view(batch_size,-1)
#loss = torch.abs(gt-output)
#loss

ValueError: need more than 2 values to unpack

In [707]:
y_batch

array([[ 1.],
       [ 0.]])

In [57]:
torch.manual_seed(0)
x = Variable(torch.rand(batch_size,4)*10)
output = torch.nn.Sigmoid()(linearLayer(x)).view(-1)
loss = torch.abs(Variable(gt_fn(x.data))-output)
print  loss

Variable containing:
 0.1381
 0.1273
[torch.FloatTensor of size 2]



In [58]:
loss.mean().backward()
optimizer.step()